# Focus Area 2 — Next-Gen Precipitation for Trends & Extremes
**Core Objective**: To compare next-generation precipitation datasets for historical trends,
anomalies, and extreme event performance, enabling NMHS to select optimal products for
climate monitoring and risk assessment.

In [ ]:
import sys
import os

## Data Extraction and Loading
1. Satellite/model daily precip NetCDFs: TAMSAT, GPM/IMERG, ERA5, CBAM → precip (mm/day) 
2. Extract GHCNd Data
3. Load TAHMO data previously extracted in session 1

### Extract TAMSAT Data (Satellite Product)

In [ ]:
!git clone https://github.com/TAMSAT/tamsat_download_extraction_api

In [ ]:
file_path = "tamsat_download_extraction_api/tamsat_download_extract_api.py"

# Read the file
with open(file_path, "r") as file:
    content = file.read()

# Replace np.float with np.float64
content = content.replace("np.float", "np.float64")

# Write back the modified content
with open(file_path, "w") as file:
    file.write(content)

print("Successfully replaced np.float with np.float64.")


In [ ]:
sys.path.append('tamsat_download_extraction_api')

from tamsat_download_extract_api import download, extract

curr_dir = os.getcwd()

# Download data
download({
 "timestep": 'monthly',
 "resolution": 0.0375,
 "start_date": '1981-01-01',
 "end_date": '2025-05-29',
 "version": 3.1,
 "localdata_dir": f"{os.path.join(curr_dir, 'tamsat_data')}"
 })

In [ ]:
kenya_w, kenya_s, kenya_e, kenya_n = bbox_from_polygon('Kenya', maps_key)

extract({
    "extract_type": 'domain',
    "S": kenya_s,
    "N": kenya_n,
    "W": kenya_w,
    "E": kenya_e,
    "timestep": 'daily',
    "start_date": '2023-01-01',
    "end_date": '2023-01-31',
    "version": 3.1,
    "localdata_dir": f"{os.path.join(curr_dir, 'tamsat_data')}",
    "resolution": 0.0375
})

## Extract IMERG Data (Satellite Product)
Using Google Earth Engine

## Extract ERA5 (Reanalysis Product)

In [ ]:
start_date = '2023-01-01T00:00:00'
end_date   = '2023-01-31T00:00:00'
region_name = 'Kenya'
era5_ke_data = era5_data_extracts(region_name, start_date, end_date, maps_key, era5_l=False)

te_ds_era5_ke = era5_var_handling(era5_ke_data, 'temperature_2m', xarray_ds=True)
pr_ds_era5_ke = era5_var_handling(era5_ke_data, 'total_precipitation', xarray_ds=True)
# ws_ds_era5_gh = era5_var_handling(era5_gh_data, 'wind_speed', xarray_ds=True)
merged_era5_ke = xr.merge([te_ds_era5_ke, pr_ds_era5_ke])
# sort the latitudes
merged_era5_ke = merged_era5_ke.sortby('lat')
# sort the longitudes
merged_era5_ke = merged_era5_ke.sortby('lon')

In [ ]:
plot_xarray_data2(
    xarray_ds=merged_era5_ke,
    fig_title="ERA5 Temperature and Precipitation Analysis in Kenya",
    columns=["temperature_2m", "total_precipitation"],
    plot_size=6,
    region_name="Kenya",
    maps_key=maps_key
)

## Extract CBAM Data

In [ ]:
# Define parameters as shown in the docs
product = 'cbam_historical_analysis'
attribs = ["max_temperature", "min_temperature", "total_rainfall"]
start_date = '2012-01-01'
end_date = '2023-12-31'
bbox = bbox_from_polygon("Dar es Salaam", maps_key=maps_key)
print(bbox)

# Create a client and extract data
client = CBAMClient('/content/Safe4All-workshop/config.json')
cbam_data = extract_cbam_data(start_date=start_date, end_date=end_date,
                         attributes=attribs, cbam_client=client, bbox=[39.0, -7.2, 39.5, -6.6],
                         output_type='csv')


## Data Processing
1. Build country-wide LTN (e.g., 2014–2025) and anomalies.
2. Compare skill of multiple rainfall products against ground for events vs climatology.

3. Only use Ground stations with high confidence as determined in Focus area 1
4. Rank products by RMSE, bias, correlation; articulate operational pros/cons.
5. (New) Add significance tests for trends and visualize spatial biases.